In [1]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'font.family': 'serif', 'mathtext.fontset': 'dejavuserif',
                 'font.size': 12, 'text.latex.preamble': r"\usepackage{amsmath}",
                 'xtick.major.pad': 2, 'ytick.major.pad': 2, 'xtick.major.size': 6, 'ytick.major.size': 6,
                 'xtick.minor.size': 3, 'ytick.minor.size': 3, 'axes.linewidth': 2, 'axes.labelpad': 1})

# from common.Tools
def format_axis(ax: mpl.axes._axes.Axes) -> None:
    ax.minorticks_on(); ax.grid(visible=True, which='major', linestyle=':')
    ax.tick_params(axis='both', which='both', direction='out')
    ax.xaxis.set_ticks_position('both')
    ax.yaxis.set_ticks_position('both')
    ax.patch.set_alpha(0.0)

In [2]:
import pandas as pd
import numpy as np
from astropy import constants as const, units as u
from scipy.optimize import curve_fit, fsolve

In [3]:
COLUMNS = ['KIC', 'Mass', 'S_Mass', 'Radius', 'S_Radius', 'Logg_Seis', 'S_Logg_Seis', 'Teff', 'S_Teff',
           '[Fe/H]', 'S_[Fe/H]', '[Alp/Fe]', 'S_[Alp/Fe]', '[C/Fe]', 'S_[C/Fe]', '[N/Fe]', 'S_[N/Fe]', 'Alpha_Cat']

df = pd.read_table('Table4.data', sep='\s+', low_memory=False)
df = df[((df['Evol_State'] == 'RGB') | (df['Evol_State'] == 'RGB_AGB')) & (df['Cat_Tab'] == 'Gold')]
df = df[(df['Mass'] > -9999.0) & (df['Radius'] > -9999.0)]  # filter out bad values

# df['Mass'] /= df['F_Numax']
df = df[COLUMNS]
df.set_index('KIC', inplace=True)
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 4997 entries, 893214 to 12885196
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Mass         4997 non-null   float64
 1   S_Mass       4997 non-null   float64
 2   Radius       4997 non-null   float64
 3   S_Radius     4997 non-null   float64
 4   Logg_Seis    4997 non-null   float64
 5   S_Logg_Seis  4997 non-null   float64
 6   Teff         4997 non-null   float64
 7   S_Teff       4997 non-null   float64
 8   [Fe/H]       4997 non-null   float64
 9   S_[Fe/H]     4997 non-null   float64
 10  [Alp/Fe]     4997 non-null   float64
 11  S_[Alp/Fe]   4997 non-null   float64
 12  [C/Fe]       4997 non-null   float64
 13  S_[C/Fe]     4997 non-null   float64
 14  [N/Fe]       4997 non-null   float64
 15  S_[N/Fe]     4997 non-null   float64
 16  Alpha_Cat    4995 non-null   object 
dtypes: float64(16), object(1)
memory usage: 702.7+ KB


In [4]:
for col in COLUMNS:
    if col in ['KIC', 'Alpha_Cat']: continue
    print(f'{col}: {df[col].min()}, {df[col].max()}')

Mass: 0.5193, 3.2319
S_Mass: 0.0217, 0.1775
Radius: 3.893, 52.9843
S_Radius: 0.0664, 0.9545
Logg_Seis: 1.278, 3.258
S_Logg_Seis: 0.0035, 0.0128
Teff: 3892.5522, 5259.778
S_Teff: 32.9835, 176.1391
[Fe/H]: -9999.0, 0.5205
S_[Fe/H]: 0.058, 0.058
[Alp/Fe]: -9999.0, 0.3728
S_[Alp/Fe]: 0.022, 0.1873
[C/Fe]: -9999.0, 0.3404
S_[C/Fe]: -9999.0, 0.491
[N/Fe]: -9999.99, 1.5659
S_[N/Fe]: -9999.0, 5.9129


# Sample Selection Visualization

In [5]:
fig, axs = plt.subplots(3, 1, figsize=(5.6, 9.6))  # (6.4, 9.6), 12/15/2024

# Step 1: Select alpha-poor stars.
ax = axs[0]
n0 = len(df)  # Number of Gold RGB stars.

cmap = mpl.colormaps['cool_r']
norm = mpl.colors.Normalize(vmin=0.9, vmax=1.8)  # 12/19/2024
ax.scatter(df['[Fe/H]'], df['[Alp/Fe]'], s=5, c=cmap(norm(df['Mass'])), alpha=0.5)  # , rasterized=True)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             ax=ax, orientation='vertical', label='$M/\mathrm{{M}}_{{\odot}}$')

df = df[df['Alpha_Cat'] == 'Apoor']
df.drop(columns='Alpha_Cat', inplace=True)
n1 = len(df)  # Number of alpha-poor stars.

ax.plot([-0.9, -0.4, 0.2, 0.5], [0.14, 0.14, 0.05, 0.05], 'r--', lw=1.5)
ax.set_xlim(-0.9, 0.5)
ax.set_ylim(-0.05, 0.35)

ax.set_xlabel(r'$[{\rm Fe}/{\rm H}]$')
ax.set_ylabel(r'$[\alpha/{\rm Fe}]$')
ax.set_title(rf'${n0}$ Gold RGB stars $\Rightarrow$ ${n1}$ $\alpha$-poor stars')
format_axis(ax)

In [6]:
# Derive mass percentiles. (12/22/2024)

FeH_fine = np.linspace(-0.45, 0.45, 181)
pct_5th = np.zeros_like(FeH_fine)
pct_95th = np.zeros_like(FeH_fine)

for i, FeH in enumerate(FeH_fine):
    df_FeH = df[(df['Mass'].between(0.9, 1.8)) &
                (df['[Fe/H]'] > max(FeH - 0.1, -0.45)) &
                (df['[Fe/H]'] < min(FeH + 0.1, 0.45))]
    pct_5th[i] = np.percentile(df_FeH['Mass'], 5)
    pct_95th[i] = np.percentile(df_FeH['Mass'], 95)

In [7]:
# from scipy.signal import savgol_filter
# pct_5th = savgol_filter(pct_5th, window_length=11, polyorder=2)
# pct_95th = savgol_filter(pct_95th, window_length=11, polyorder=2)

with open('Table4_mass_percentiles.npz', 'wb') as f:
    np.savez(f, FeH_fine=FeH_fine, pct_5th=pct_5th, pct_95th=pct_95th)

# f = np.load('Table4_mass_percentiles.npz')
# FeH_fine, pct_5th, pct_95th = f['FeH_fine'], f['pct_5th'], f['pct_95th']

In [8]:
# Step 2: Apply mass and metallicity cuts.
# fig, ax = plt.subplots(figsize=(6.4, 4.8))
ax = axs[1]

cmap = mpl.colormaps['Wistia']
norm = mpl.colors.Normalize(vmin=-0.05, vmax=0.15)
ax.scatter(df['Mass'], df['[Fe/H]'], s=5, c=cmap(norm(df['[Alp/Fe]'])), alpha=0.5)  # , rasterized=True)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             ax=ax, orientation='vertical', label=r'$[\alpha/{\rm Fe}]$')

for Mass in [0.90, 1.80]: ax.axvline(Mass, color='b', ls='--', lw=1.5)
for FeH in [-0.45, 0.45]: ax.axhline(FeH, color='b', ls='--', lw=1.5)
ax.plot(pct_5th, FeH_fine, 'g:', lw=1.5)
ax.plot(pct_95th, FeH_fine, 'g:', lw=1.5)
ax.set_xlim(0.7, 2.0)
ax.set_ylim(-0.5, 0.5)

df = df[(df['Mass'] > 0.90) & (df['Mass'] < 1.80) & (df['[Fe/H]'] > -0.45) & (df['[Fe/H]'] < 0.45)]
n2 = len(df)  # Number of stars within mass and metallicity ranges.

ax.set_xlabel('$M/\mathrm{{M}}_{{\odot}}$')
ax.set_ylabel(r'$[{\rm Fe}/{\rm H}]$')
ax.set_title(rf'$\Rightarrow$ ${n2}$ stars within $M$ and $[{{\rm Fe}}/{{\rm H}}]$ ranges')
format_axis(ax)

In [9]:
# Step 3: Select stars for RGB calibration.
# fig, ax = plt.subplots(figsize=(6.4, 4.8))
ax = axs[2]
n3 = np.count_nonzero(df['Teff'] + df['Logg_Seis'] * 1000 > 7500)  # Number of stars for RGB calibration.

cmap = mpl.colormaps['winter']
norm = mpl.colors.Normalize(vmin=-0.45, vmax=0.45)
ax.scatter(df['Teff'], df['Logg_Seis'], s=5, c=cmap(norm(df['[Fe/H]'])), alpha=0.5)  # , rasterized=True)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             ax=ax, orientation='vertical', label='[Fe/H]')

ax.plot([4000, 5500], [3.5, 2.0], 'r--', lw=1.5)
ax.set_xlim(5200, 3800)
ax.set_ylim(3.3, 1.3)

ax.set_xlabel(r'$T_{\rm eff}$ [K]')
ax.set_ylabel(r'$\log g$ [cm s$^{-2}$]')
ax.set_title(rf'$\Rightarrow$ ${n3}$ lower RGB stars (below RGBB)')  # 12/19/2024
format_axis(ax)

fig.tight_layout()
fig.savefig('selection.pdf', bbox_inches='tight')
plt.close()
# plt.show()

# Correlated C and N errors

In [10]:
# Directly from 2024.10.16 APOKASC3_Table4_explorer.ipynb
# From 2023.11.28 APOKASC3_FDU_and_RGBB.ipynb.

def model(x, a, b1, b2, c1, c2, d):
    '''
    [*/*] = A + B1 (M-1) + B2 (M-1)^2 + C1 [Fe/H] + C2 [Fe/H]^2 + D (M-1) [Fe/H]

    '''

    return a + b1*(x[0]-1.0) + b2*(x[0]-1.0)**2 + c1*x[1] + c2*x[1]**2 + d*(x[0]-1.0)*x[1]

In [11]:
# From 2023.11.28 APOKASC3_FDU_and_RGBB.ipynb, customized.

df_ = df[df['Teff'] + df['Logg_Seis'] * 1000 > 7500].copy()

def explore_ratio(CATALOG=df_, ratio='[C/N]', rho_CN=None):
    xdata = np.array(CATALOG[['Mass', '[Fe/H]']]).T
    ydata = np.array(CATALOG[ratio])
    popt, pcov = curve_fit(model, xdata, ydata)
    # print(popt, pcov, sep='\n')

    ypred = model(xdata, *popt)
    disc = ydata - ypred
    if ratio == '[C/N]' and rho_CN is not None:
        sigma = np.sqrt(np.square(df_['S_[C/Fe]']) + np.square(df_['S_[N/Fe]'])
                        - 2 * rho_CN * df_['S_[C/Fe]'] * df_['S_[N/Fe]'])
        chi1 = disc / sigma
    else:
        chi1 = disc / CATALOG[f'S_{ratio}']
    chi2 = np.sum(np.square(chi1)) / (len(CATALOG)-6)

    return chi2

In [12]:
chi2_CFe = explore_ratio(ratio='[C/Fe]')
print(f'{chi2_CFe = }')
df_['S_[C/Fe]'] = df_['S_[C/Fe]'] * np.sqrt(chi2_CFe)
print('rescaled:', explore_ratio(ratio='[C/Fe]'))

chi2_CFe = 9.167845286697043
rescaled: 1.0


In [13]:
chi2_NFe = explore_ratio(ratio='[N/Fe]')
print(f'{chi2_NFe = }')
df_['S_[N/Fe]'] = df_['S_[N/Fe]'] * np.sqrt(chi2_NFe)
print('rescaled:', explore_ratio(ratio='[N/Fe]'))

chi2_NFe = 7.3491758798855145
rescaled: 1.0000000000000002


In [14]:
df_['[C/N]'] = df_['[C/Fe]'] - df_['[N/Fe]']
df_['S_[C/N]'] = np.sqrt(np.square(df_['S_[C/Fe]']) + np.square(df_['S_[N/Fe]']))
chi2_CN = explore_ratio(ratio='[C/N]')
print(f'{chi2_CN = }')

chi2_CN = 1.0940750786137188


In [15]:
func_chi2 = lambda rho: explore_ratio(ratio='[C/N]', rho_CN=rho) - 1
rho_opt = fsolve(func_chi2, -0.33)[0]
rho_opt

-0.09457213483169677

In [16]:
if False:  # 12/20/2024
    RES = 31
    rho_arr = np.linspace(-0.7, 0.3, RES)
    chi2_arr = [explore_ratio(ratio='[C/N]', rho_CN=rho) for rho in rho_arr]

    fig, ax = plt.subplots(figsize=(6.4, 4.8))

    ax.plot(rho_arr, chi2_arr, 'k-')
    ax.set_xlabel(r'$\rho_{\rm C,N}$')  # 'RHO_CN'
    ax.set_ylabel(r'Reduced $\chi^2 ([{\rm C}/{\rm N}])$')  # 'REDUCED_CHI2'

    ax.axvline(rho_opt, ls='--', c='b',
            label=rf'Optimal $\rho_{{\rm C,N}} = {rho_opt:.4f}$')  # f'Optimal: {rho_opt=:.6f}'
    ax.legend()
    ax.set_title(r'Measuring correlation between ${\rm C}$ and ${\rm N}$ errors')

    format_axis(ax)
    fig.tight_layout()
    fig.savefig('CN_errors.pdf', bbox_inches='tight')
    plt.close()
    # plt.show()

In [17]:
def rescale_CN_errors(df: pd.core.frame.DataFrame):
    df['S_[C/Fe]'] = df['S_[C/Fe]'] * np.sqrt(chi2_CFe)
    df['S_[N/Fe]'] = df['S_[N/Fe]'] * np.sqrt(chi2_NFe)
    df['[C/N]'] = df['[C/Fe]'] - df['[N/Fe]']
    df['S_[C/N]'] = np.sqrt(np.square(df['S_[C/Fe]']) + np.square(df['S_[N/Fe]'])
                            - 2 * rho_opt * df['S_[C/Fe]'] * df['S_[N/Fe]'])

rescale_CN_errors(df)
# From 2024.08.28 APOKASC3_Table4_explorer.ipynb.
df.to_csv('Table4_Apoor_RGB_Gold.csv')  # , index=False

# The RGB+RC Gold sample

In [18]:
# From 2024.11.13 APOKASC3_Table4_explorer.ipynb.

COLUMNS = ['KIC', 'Evol_State', 'Mass', 'S_Mass', 'Radius', 'S_Radius', 'Logg_Seis', 'S_Logg_Seis', 'Teff', 'S_Teff',
           '[Fe/H]', 'S_[Fe/H]', '[Alp/Fe]', 'S_[Alp/Fe]', '[C/Fe]', 'S_[C/Fe]', '[N/Fe]', 'S_[N/Fe]', 'Alpha_Cat']

df = pd.read_table('Table4.data', sep='\s+', low_memory=False)
df['Evol_State'] = df['Evol_State'].replace({'RGB_AGB': 'RGB'})
df = df[((df['Evol_State'] == 'RGB') | (df['Evol_State'] == 'RC')) & (df['Cat_Tab'] == 'Gold')]
df = df[(df['Mass'] > -9999.0) & (df['Radius'] > -9999.0) & (df['[Fe/H]'] > -9999.0) &
        (df['[Alp/Fe]'] > -9999.0) & (df['[C/Fe]'] > -9999.0) & (df['[N/Fe]'] > -9999.0)]  # filter out bad values

# df['Mass'] /= df['F_Numax']
df = df[COLUMNS]
df.set_index('KIC', inplace=True)
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 9878 entries, 893214 to 12885196
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Evol_State   9878 non-null   object 
 1   Mass         9878 non-null   float64
 2   S_Mass       9878 non-null   float64
 3   Radius       9878 non-null   float64
 4   S_Radius     9878 non-null   float64
 5   Logg_Seis    9878 non-null   float64
 6   S_Logg_Seis  9878 non-null   float64
 7   Teff         9878 non-null   float64
 8   S_Teff       9878 non-null   float64
 9   [Fe/H]       9878 non-null   float64
 10  S_[Fe/H]     9878 non-null   float64
 11  [Alp/Fe]     9878 non-null   float64
 12  S_[Alp/Fe]   9878 non-null   float64
 13  [C/Fe]       9878 non-null   float64
 14  S_[C/Fe]     9878 non-null   float64
 15  [N/Fe]       9878 non-null   float64
 16  S_[N/Fe]     9878 non-null   float64
 17  Alpha_Cat    9878 non-null   object 
dtypes: float64(16), object(2)
memory usage: 1.4+

In [19]:
for col in COLUMNS:
    if col in ['KIC', 'Alpha_Cat']: continue
    print(f'{col}: {df[col].min()}, {df[col].max()}')

Evol_State: RC, RGB
Mass: 0.5193, 3.649
S_Mass: 0.0217, 0.1775
Radius: 3.893, 52.9843
S_Radius: 0.0664, 0.9545
Logg_Seis: 1.278, 3.258
S_Logg_Seis: 0.0034, 0.0128
Teff: 3892.5522, 5311.763
S_Teff: 32.9835, 176.1391
[Fe/H]: -2.3446, 0.5205
S_[Fe/H]: 0.058, 0.058
[Alp/Fe]: -0.0676, 0.3728
S_[Alp/Fe]: 0.022, 0.1873
[C/Fe]: -0.8748, 0.4763
S_[C/Fe]: 0.0068, 0.1543
[N/Fe]: -0.5754, 0.7966
S_[N/Fe]: 0.0081, 5.9129


In [20]:
# Step 1: Select alpha-poor stars.
n0 = len(df)  # Number of Gold RGB stars.
df = df[df['Alpha_Cat'] == 'Apoor']
df.drop(columns='Alpha_Cat', inplace=True)
n1 = len(df)  # Number of alpha-poor stars.

print(n0, '->', n1)

9878 -> 7907


In [21]:
# Step 2: Apply mass and metallicity cuts.
df = df[(df['Mass'] > 0.90) & (df['Mass'] < 1.80) & (df['[Fe/H]'] > -0.45) & (df['[Fe/H]'] < 0.45)]
n2 = len(df)  # Number of stars within mass and metallicity range.

print(n1, '->', n2)

7907 -> 6603


In [22]:
rescale_CN_errors(df)
df.to_csv('Table4_Apoor_RGB+RC_Gold.csv')  # index=False